<a href="https://colab.research.google.com/github/nihermann/Pokemaenner/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##################################### Console #####################################
# !git clone https://github.com/nihermann/Pokemaenner.git
# %cd Pokemaenner/
# !git status
###################################################################################

Cloning into 'Pokemaenner'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 134 (delta 68), reused 70 (delta 28), pack-reused 0
Receiving objects: 100% (134/134), 496.52 KiB | 12.11 MiB/s, done.
Resolving deltas: 100% (68/68), done.
/content/Pokemaenner
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title # Using Gan to create new Pokemon
import tensorflow as tf
from manager import GANManager
from data import DataGenerator
import GAN

In [ ]:
#@title ## Data Settings
image_shape = (256, 256) #@param
image_path = "/content/drive/MyDrive/images/" #@param {type:"string"}
batch_size = 32 #@param {type:"integer"}
validation_split = 0.1 #@param {type:"slider", min:0, max:0.5, step:0.1}
shuffle = True #@param {type:"boolean"}

data = DataGenerator(
    img_path=image_path,
    batch_size=batch_size,
    img_height=image_shape[0],
    img_width=image_shape[1],
    validation_split=validation_split,
    shuffle=shuffle
)

Found 10135 files belonging to 14 classes.
Using 9122 files for training.
Found 10135 files belonging to 14 classes.
Using 1013 files for validation.


In [ ]:
#@title Generator Arguments
latentspace = 250 #@param {type:"slider", min:2, max:1000, step:1}


generator = GAN.Generator(
    latentspace=latentspace
)

In [ ]:
#@title ## Discriminator Arguments
discriminator = GAN.Discriminator(
    input_shape=(None, image_shape[0], image_shape[1], 4)
)

In [ ]:
#@title ## Hyperparameters


loss_function = "Binary Cross Entropy" #@param ["Binary Cross Entropy", "Mean Squared Error"]
optimizer = "Adam" #@param ["Adam", "RMSprop", "SGD"]
learning_rate = 0.001 #@param {type:"number"}


## Dropdown equivalents
loss_functions = {
    "Binary Cross Entropy": tf.keras.losses.BinaryCrossentropy(),
    "Mean Squared Error": tf.keras.losses.MSE
}

optimizers = {
    "Adam": tf.keras.optimizers.Adam(learning_rate),
    "RMSprop": tf.keras.optimizers.RMSprop(learning_rate),
    "SGD": tf.keras.optimizers.SGD
}


## Final
kwargs = {
    "batch_size": batch_size,
    "loss": loss_functions[loss_function],
    "optimizer": optimizers[optimizer]
}

manager = GANManager(
    kwargs=kwargs,
    generator=generator,
    discriminator=discriminator,
    data=data
)

In [ ]:
#@title # Training Parameters
epochs = 100 #@param {type:"integer"}
samples_per_epoch = 10000 #@param {type:"integer"}
trainings_frequency =  (5, 5)#@param {type:"raw"}
print_every =  1#@param {type:"integer"}
print_verbose = True #@param {type:"boolean"}
save_pictures_every = 4 #@param {type:"integer"}
how_many_pictures_to_save = 10 #@param {type:"integer"}
save_model_every = 0 #@param {type:"integer"}


manager.train(
    epochs=epochs,
    samples_per_epoch=samples_per_epoch,
    trainings_frequency=trainings_frequency,
    print_every=print_every,
    print_verbose=print_verbose,
    save_pictures_every=save_pictures_every,
    how_many_pictures_to_save=how_many_pictures_to_save,
    save_model_every=save_model_every
    )

|T| ---------------------------------------------------------------> |V| -------------------------------->
|D| Mean G-Loss: 12.757805824279785, Mean D-Loss: 0.005183156114071608, Mean D-Loss for real images: 0.006516627036035061, Mean D-Loss for generated images: 0.0  Mean D-Accuracy for real images: 0, Mean D-Accuracy for generated images: 0 || 
|T| ---------------------------------------------------------------> |V| -------------------------------->
|D| Mean G-Loss: 15.424947738647461, Mean D-Loss: 1.2884834177384619e-07, Mean D-Loss for real images: 0.0003814099763985723, Mean D-Loss for generated images: 0.0  Mean D-Accuracy for real images: 0, Mean D-Accuracy for generated images: 0 || 
|T| ---------------------------------------------------------------> |V| -------------------------------->
|D| Mean G-Loss: 15.424947738647461, Mean D-Loss: 1.024165641183572e-08, Mean D-Loss for real images: 0.00012278366193640977, Mean D-Loss for generated images: 0.0  Mean D-Accuracy for real im